# Lab 9. Neural networks

#### Table of contents

1. Overview
2. The QM7 dataset
3. Object oriented programming
4. Prepare the data
5. Feedforward
6. Backpropagation
7. Debuging
8. Training


## 1. Overview

In this lab session we will write a code to optimize a neural network, run the model to predict the atomization energy of some small molecules and select the best hyperparameters.

## 2. The QM7 dataset

This dataset is an extension of the QM7 dataset for multitask learning where 13 additional properties (e.g. polarizability, HOMO and LUMO eigenvalues, excitation energies) have to be predicted at different levels of theory (ZINDO, SCS, PBE0, GW). Additional molecules comprising chlorine atoms are also included, totalling 7211 molecules.
The dataset is composed of two multidimensional arrays $X$ ($7211\times 23\times 23$) and $T$ ($7211\times 14$) representing the inputs (Coulomb matrices) and the labels (molecular properties) and one array names of size 14 listing the names of the different properties.
More details are provided in this [paper](https://iopscience.iop.org/article/10.1088/1367-2630/15/9/095003/meta).

Basically, the datatset contains features to describe some small molecules (these features are called Coulomb matrices) and various molecular properties (14) as follow:

1. Atomization energies (PBE0, unit: kcal/mol)
2. Excitation of maximal optimal absorption (ZINDO, unit: eV)
3. Absorption Intensity at maximal absorption (ZINDO)
4. Highest occupied molecular orbital HOMO (ZINDO, unit: eV)
5. Lowest unoccupied molecular orbital LUMO (ZINDO, unit: eV)
6. First excitation energy (ZINDO, unit: eV)
7. Ionization potential IP (ZINDO, unit: eV)
8. Electron affinity EA (ZINDO, unit: eV)
9. Highest occupied molecular orbital HOMO (PBE0, unit: eV)
10. Lowest unoccupied molecular orbital LUMO (PBE0, unit: eV)
11. Highest occupied molecular orbital HOMO (GW, unit: eV)
12. Lowest unoccupied molecular orbital LUMO (GW, unit: eV)
13. Polarizabilities (PBE0, unit: $A^3$)
14. Polarizabilities (SCS, unit: $A^3$)

Because these properties are complicated to compute, methods based on machine learning can be trained to predict them based on some meaningfull features. Coulomb matrices are such good representations.

A Coulmb matrix is defined based on the atomic positions $R_i$ and atomic charges $Z_i$ of atoms in a molecule as:

$M_{IJ}=\left\{
\begin{array}{ll}
0.5Z_I^{2.4}\text{ for }I=J\\
\frac{Z_IZ_J}{|R_I-R_J|}\text{ for }I\neq J\\
\end{array}
\right.
$

Here, the Coulomb matrices are already computed and provided in the training set.

## 3. Object oriented programming

In Python like in many other object oriented programmation languages, you often work with classes. Classes are somewhat similar to functions but need to be instantiated as objects before you can use them. Most of the libraries we have used previously are actually classes (e.g. Numpy, StandardScaler, LinearRegression, etc.). Classes contain functions (called methods) and variables that can be accesses by the object instance.

One of the main advantage with classes is that some key variables are accessible anywhere from all methods without having to pass it as arguments. In the neural network example below, these key variables are the architecture of the network, the number of layers, and weights and biases. 

To complete the lab succesfully, you just need to know some basics about classes. The only differences between classes/methods and the functions we usually define are the following:

- A class must have a default method `__init__` to define the key variables
- Some other methods are defined within a class
  - All these methods take as 1st argument the keyword `self`
  - To call any method from another method in the class the name must be provided with the prefix `self.` and you can ignore the `self` argument

That's all you need to know to complete the lab.
You can learn more about object oriented programming and the use of classes in [this Wikipedia article](https://en.wikipedia.org/wiki/Object-oriented_programming).

## 4. Prepare the data

Let's first load the data and reshape it into 2D arrays (this was explained in Lab8). 

In [1]:
from scipy.io import loadmat
qm7 = loadmat('qm7b.mat')

In [2]:
import numpy as np
X0 = qm7['X']
X = X0.reshape(7211,529)
X = np.c_[X]
print(X.shape)

(7211, 529)


In [3]:
y = qm7['T'][:,0]*0.043
y = np.c_[y]
print(y.shape)

(7211, 1)


You will now answer the following questions in the code block below that defines the entire class `Network` as well as some additional (regular) functions. Because this might be complicated for you to debug, once you have completed the feedforward method (Q.1), you can jump to section 7 and test you feedforward method to be sure it works. You can proceed similarly with the backpropagation method (once you answered Q.2, Q.3 and Q.4 you can jump to section 7 to test your backprop method). Although the following questions only address two functions of the class `Network` you must read the whole code (and all comments) carefully to be sure understand it perfectly!

## 5. Feedforward

__Q.1.__ Complete the `feedforward` method (Q1). The loop provides for each layer (from $l=2$ to $l=L-1$) the corresponding weight matrix $\omega$ and the bias vector $b$. The loop starts at $l=2$ because there is no weights and biases in the input layer, and it ends at $l=L-1$ because we will use a linear activation in the output layer $l=L$ (see part of the code after the loop). Inside the loop, you need to compute the function $z = \omega\cdot a+b$ and append the result (assigned to the variable `z`) to the list `z_list`. You can use numpy's library for dot product ($a\cdot b=$`np.dot(a,b)`). Then you have to pass the result throught the activation function, here we will use the hyperbolic tangeant, and append the result (assigned to the variable `a`) to the list `a_list` (2 marks).

## 6. Backpropagation

__Q.2.__ Complete the `backprop` method (Q2) to evaluate the error in the output layer. You will assign this error to the variable `delta`. This corresponds to equation BP1: $\delta^L = \frac{\partial J}{\partial a^L}\sigma'(z^L)$. However, since the activation in the last layer is linear, BP1 becomes: $\delta^L = \frac{\partial J}{\partial a^L}$. The derivative of $J$ with respect to activation is provided as the method `J_prime`. You just need to evaluate `J_prime` with arguments the activation of the output layer and the actual output value `yi`. Note that because we are inside a class, you must call functions with the `self.` prefix and you can ignore the `self` argument. You can see as an example how the `backprop` function is called within the `update_mini_batch` method. In Python, you can access values in a list (or an array) starting from the end. For example, `a[-1]` corresponds to the last value in the list `a`, `a[-2]` correspond to the second last value, etc. You can use this trick to access the output activation which corresponds to the last value of the `a_list` (2 marks).

__Q.3.__ Complete the `backprop` method (Q3) to evaluate the rate change of the cost with respect to biases in the output layer. The array `nabla_b` contains the partial derivatives $\frac{\partial J}{\partial b^l}$ for each layer $l$. Here you must assign only the last value of the list `nabla_b` corresponding to the equation BP3: $\frac{\partial J}{\partial b^L} = \delta^L$ for $l=L$. Note that the value $\delta^L$ was computed in the previous question. You can use negative index to assign the last value of the `nabla_b` list (2 marks).

__Q.4.__ Complete the `backprop` method (Q4) to evaluate the rate change of the cost with respect to weights in the last layer. The array `nabla_w` contains the partial derivatives $\frac{\partial J}{\partial \omega^l}$ for each layer $l$. Here you must assign the last value of the list `nabla_w` corresponding to the equation BP4: $\frac{\partial J}{\partial \omega^L} = \delta^L\cdot(a^{L-1})^T$ for $l=L$. The values $\delta^L$ and $a$ were computed previously. You must take the transpose of the array $a^{L-1}$. This can be achieved with the `.transpose()` or `.T` suffix. You can use negative indices to assign the last value of the `nabla_w` list (2 marks).

In [4]:
import random
import numpy as np
from numpy.random import RandomState
import time

class Network(object):

    def __init__(self, sizes):
        prng = RandomState(33) # seed for random numbers
        self.num_layers = len(sizes)        
        self.sizes = sizes
        self.biases = [prng.randn(y, 1) for y in sizes[1:]]
        self.weights = [prng.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
                
    def feedforward(self, a):  
        # The list of activations
        a_list = [a]
        # The list of z's
        z_list = []
        # We loop over biased and weights in each layer (from the 2nd layer to L-1)
        # Variables b correspond to the bias vector in a given layer
        # Variables w correspond to the weight matrix in a given layer
        for b, w in zip(self.biases[:-1], self.weights[:-1]):
            ### Q1. BEGIN SOLUTION
            ### Q1. END SOLUTION
        # Now we compute activation and z of the output layer
        z = np.dot(self.weights[-1], a)+self.biases[-1]
        z_list.append(z)
        # The output layer is linear (no activation) therefore a = z
        a_list.append(z)
        return a_list,z_list

    def SGD(self, X, y, hyper_params):
        # We get the hyper-parameters
        epochs, mini_batch_size, alpha = hyper_params
        # This computes the RMSE
        # Also returns an array of the output values of the network
        rmse, y_pred = self.evaluate(X,y)
        print("Epoch {:3d} complete {:.4f} eV".format(0,rmse))
        m,n = X.shape
        rmse_list = []
        # Loop over epochs
        for j in range(epochs):
            t0 = time.time()
            # Compute number of batches
            total_batch = int(m/mini_batch_size)
            # Loop over batches
            for k in range(total_batch):
                offset = k*mini_batch_size
                Xi = X[offset:offset+mini_batch_size]
                Yi = y[offset:offset+mini_batch_size]
                # Update weights and biases
                self.update_mini_batch(Xi,Yi,alpha)
            if (j+1) % 1 == 0:
                rmse, y_pred = self.evaluate(X,y)
                rmse_list.append(rmse)
                t = time.time()
                print("Epoch {:3d} complete {:.4f} eV @{:.3f}s".format(j+1,rmse,t-t0))
            else: 
                t = time.time()
                print("Epoch {:3d} complete @{:.3f}s".format(j+1,t-t0))
        return rmse_list, y_pred

    def update_mini_batch(self, Xi, Yi, alpha):
        # Create arrays filled with zeros
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        mi,ni = Xi.shape
        # Loop over examples in the mini batch
        for i in range(mi):
            # Backprop
            delta_nabla_b, delta_nabla_w = self.backprop(np.c_[Xi[i]], Yi[i])
            # Compute partial derivatives over single example
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        # Update weights and biases via GD based on all examples in the mini batch
        self.weights = [w-(alpha/mi)*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(alpha/mi)*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, xi, yi):
        # Initialize arrays
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # Forward path
        # We get the list of activations and z values in each layer
        a_list,z_list = self.feedforward(xi)
        # Compute delta of last layer (BP1)
        ### Q2. BEGIN SOLUTION
        ### Q2. END SOLUTION
        # Compute dJ/db of the last layer (BP3)
        ### Q3. BEGIN SOLUTION
        ### Q3. END SOLUTION
        # Compute dJ/dw of the last layer (BP4)
        ### Q4. BEGIN SOLUTION
        ### Q4. END SOLUTION
        # Backpropagate
        # Loop over layers starting at 2
        # Here we will use negative indices
        # For example -2, -3, etc. -(L+1) correspond to layers L-1, L-2, etc. 2
        for l in range(2, self.num_layers):
            z = z_list[-l]
            sp = tanh_prime(z)
            # Backpropagate delta of each layers (BP2)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            # Compute dJ/db of each layer (BP3)
            nabla_b[-l] = delta
            # Compute dJ/dw of each layer (BP4)
            nabla_w[-l] = np.dot(delta, a_list[-l-1].transpose())
        # Returns lists of arrays containing partial derivatives dJ/db and dJ/dw
        # Each list has the dimension of the number of layers-1
        # Each array has dimension of weights and biases array per layer
        return nabla_b, nabla_w
    
    def evaluate(self, X_val, y_val):
        # Feedforward and compute RMSE
        m_val,n_val = X_val.shape
        a_list,z_list = self.feedforward(np.c_[X_val].T)
        rmse = np.sqrt(np.sum((a_list[-1]-np.c_[y_val].T)**2)/m_val)
        # Returns RMSE value and list of output values over arguments data X_val/y_val
        return rmse,a_list[-1]
    
    def J_prime(self, hi, y):
        # Derivative of the cost function with respect to output activation
        # Here the cost function J is the MSE/2
        # Just for one example
        return hi-y

# Some activation functions and their respective derivatives
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def tanh(z):
    return 2*sigmoid(2*z)-1

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def tanh_prime(z):
    return 1-tanh(z)**2

## 7. Debuging

Below we define a minimal network of 1 input, 1 hidden and 1 output units that we feed with the value 1.0 to test the `feedforward` method. If you have properly coded that method, you should find the same results listed as `Your a_list/z_list` compared to that provided in the `Correct a_list/z_list` lines.

In [5]:
# Hyperparameters for the network
# Number of input neurons, number of neurons in the hidden layers and number of output neurons
n_input, n_hidden1, n_output = 1, 1, 1
arch = [n_input, n_hidden1, n_output]
# Here we define the network
net = Network(arch)
a_list, z_list = net.feedforward(1.0)

print('Your a_list:',a_list)
print('Correct a_list:',[1.0, np.array([[-0.95212792]]), np.array([[-1.05988594]])])

print('Your z_list:',z_list)
print('Correct z_list:',[np.array([[-1.85407138]]), np.array([[-1.05988594]])])
print('If you do not find similar numbers, you must review your feedforward method.')

Your a_list: [1.0, array([[-0.95212792]]), array([[-1.05988594]])]
Correct a_list: [1.0, array([[-0.95212792]]), array([[-1.05988594]])]
Your z_list: [array([[-1.85407138]]), array([[-1.05988594]])]
Correct z_list: [array([[-1.85407138]]), array([[-1.05988594]])]
If you do not find similar numbers, you must review your feedforward method.


Similarly, based on the same network defined above, we feed the backpropagation method with values (1.0,1.0). If you have properly coded the `backprop` method, you should find the same results listed as `Your nabla_b/nabla_w` compared to that provided in the `Correct nabla_b/nabla_w` lines.

In [6]:
nabla_b, nabla_w = net.backprop(np.c_[1],np.c_[1])

print('Your nabla_b:',nabla_b)
print('Correct nabla_b:',[np.array([[0.10980294]]), np.array([[-2.05988594]])])

print('Your nabla_w:',nabla_w)
print('Correct nabla_w:',[np.array([[0.10980294]]), np.array([[1.96127491]])])
print('If you do not find similar numbers, you must review your backprop method.')

Your nabla_b: [array([[0.10980294]]), array([[-2.05988594]])]
Correct nabla_b: [array([[0.10980294]]), array([[-2.05988594]])]
Your nabla_w: [array([[0.10980294]]), array([[1.96127491]])]
Correct nabla_w: [array([[0.10980294]]), array([[1.96127491]])]
If you do not find similar numbers, you must review your backprop method.


## 8. Training

You should have now a working version of the class network. 
We can now train a neural network to predict the atomization energy of the qm7 dataset.
We will try various hyperparameters for the network and select the best set by computing error on the training set.
Note that we should be splitting the dataset into training, validation and test set to propery evaluate the performance of the model however we won't do this today.
This will be explore further in the next lectures.
However, we need to scale the data because the optimization is based on SGD. Let's standardize with sklearn's standard scaler.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(np.c_[X])
X_train = X_scaler.transform(np.c_[X])
print(X_train.shape,y.shape)

We will focus on a network with 2 hidden layers. The network architecture is therefore defined by 4 variables `n_input`, `n_hidden1`, `n_hidden2`, `n_output`. We will vary these 4 variables as follow (note that the input and output layers are fixed and defined by the dataset shape). 

- X_train.shape[1], 10, 10, 1 (10 neurons in the 1st and 2nd hidden layer)
- X_train.shape[1], 50, 50, 1
- X_train.shape[1], 100, 100, 1
- X_train.shape[1], 200, 200, 1

For each network architecture, you will vary the learning rate of the SGD method as $\alpha$ = 0.01, 0.001, 0.0001.

Define the architecture of the network, the hyperparameters for SGD, and optimize each netwroks (12 total).
For each network, note in a spreadsheet the final value of the cost in eV. The default values are initialized to the first architecture and first learning rate. Note that to answer the following questions, you should not vary the other hyperparameters (i.e. number of epoch, minibatch size).

In [ ]:
# Hyperparameters for the network
# Number of input neurons, number of neurons in the hidden layers and number of output neurons
n_input, n_hidden1, n_hidden2, n_output = X_train.shape[1], 10, 10, 1
arch = [n_input, n_hidden1, n_hidden2, n_output]
# Here we define the network
net = Network(arch)
# Hyperparameters for GD
# Number of iterations, size of minibatches and learning rate
epochs, mini_batch_size, alpha = 20, 10, 1e-1
hyper_params = epochs, mini_batch_size, alpha
# We optimize the network with SGD
rmse_list,y_predict = net.SGD(X_train,y,hyper_params)

__Q.5.__ Report the optimal values of the network hyperparameters and learning rate $\alpha$ that gives the lowest final error (1 mark).

In [ ]:
### BEGIN SOLUTION
### END SOLUTION

Based on the optimized hyperparameters rerun the model optimization this time with 100 iteration of SGD (set `epochs` to 100).

We can finally plot the following:

- the RMSE (the 1st list returned by SGD) as a function of epoch number
- the prediction of the network (the 2nd list returned by SGD) against the actual energies over the entire training set

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

plt.plot(y,y_predict.T,marker = '.',lw = 0,label = 'training data')
plt.plot(y,y,color='r',ls='-',label='y=x')
plt.legend()
plt.ylabel('energies (eV)',fontsize=22)
plt.xlabel('predicted energies (eV)',fontsize=22)
plt.tight_layout()
plt.show()

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

plt.plot(rmse_list,marker = '.',color='b',ms=20,ls='-')
plt.ylabel('predicted energies (eV)',fontsize=22)
plt.xlabel('iterations',fontsize=22)
plt.tight_layout()
plt.show()

__Q.6.__ Based on the last optimization, do you think the model overfit the data? answer by writing `yes` or `no` in the markdown below (1 mark).